In [2]:
import tensorflow as tf
import os,random
import numpy as np

def buildSmallerTrainData(xTrain, y_train, size):
    X = []
    y = []
    seen = []
    while len(seen) < size / 2:
        rand = random.randrange(0,len(xTrain))
        if rand not in seen and y_train[rand] == 0:
            X.append(xTrain[rand])
            y.append(y_train[rand])
            seen.append(rand)
    seen = []      
    while len(seen) < size / 2:
        rand = random.randrange(0,len(xTrain))
        if rand not in seen and y_train[rand] == 1:
            X.append(xTrain[rand])
            y.append(y_train[rand])
            seen.append(rand)
    return np.array(X), np.array(y)

def buildSmallerTestingData(xTrain, y_train, size, interval):
    X = []
    y = []
    seen = []
    rand = interval
    while len(seen) < size / 2:
        if rand not in seen and y_train[rand] == 0:
            X.append(xTrain[rand])
            y.append(y_train[rand])
            seen.append(rand)
        rand += 1
    rand = interval
    seen = []      
    while len(seen) < size / 2:
        if rand not in seen and y_train[rand] == 1:
            X.append(xTrain[rand])
            y.append(y_train[rand])
            seen.append(rand)
        rand += 1
    return np.array(X), np.array(y)


def readFromFile(path):
    with open(path, 'r') as content_file:
        content = content_file.read()
        return content


def isfloat(value):
    try:
        float(value)
        return True
    except ValueError:
        return False


def convert_stringArrays_to_floatArray(array):
    intArray = []

    for k in array:
        if isfloat(k):
            intArray.append(float(k))
    return intArray


def buil_db(array, value):
    data_X = []
    data_Y = []
    n = len(array)
    for i in range(0, n):
        data_X.append(array[i])
        data_Y.append(value)

    return data_X, data_Y


def readpktHistDirectFromFile(path):
    X_data = []
    files = os.listdir(path)
    for f in files:
        hist = convert_stringArrays_to_floatArray(readFromFile(path + f).split(" "))
        all_pkt =sum(hist)
        
        X_data.append(np.array(hist))
  
    return X_data


def readDataset(value,path):
    x_train, y_train = [], []
    dataset = readpktHistDirectFromFile(path=path)
    data_x, data_y = buil_db(dataset, value)
    x_train.extend(data_x)
    y_train.extend(data_y)

    return x_train, y_train

def read_train_test():
    trainin_dataset_X = []
    trainin_dataset_y = []
    X_test=[]
    y_test=[]
    names = ['caida','httpMultipleTab/4/all', 'httpMultipleTab/2/all', 'httpMultipleTab/3/all', 'httpMultipleTab/1/all',
             
             'noisyBitMultipleTab/4/all', 'noisyBitMultipleTab/2/all', 'noisyBitMultipleTab/3/all',
             'noisyBitMultipleTab/1/all', 'bitcoinCaidaNoise', 'bitcoin']
  

    labels = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
    i = 0
    for n in names:

        x_temp,y_temp = readDataset(path='/home/fatemeh/jan 19/hist/train/' + n + "/"
                                               ,value=labels[i])
        trainin_dataset_X.extend(x_temp)
        trainin_dataset_y.extend(y_temp)

        x_temp2, y_temp2 = readDataset(path='/home/fatemeh/jan 19/hist/test/' + n + "/"
                                              ,value=labels[i] )
        print("read data: ",n)
       
        X_test.extend(x_temp2)
        y_test.extend(y_temp2)
        i += 1
    
 
    return (trainin_dataset_X,trainin_dataset_y),(X_test,y_test)


In [3]:
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense,Flatten, BatchNormalization,Dropout,Conv1D,MaxPooling1D,Dropout
from keras.layers.core import Activation
from keras.utils import plot_model
import time

(x_train, y_train), (x_test, y_test) = read_train_test()
XTrainCopy, yTrainCopy = np.array(x_train), np.array(y_train)
xTestCopy, yTestCopy = np.array(x_test), np.array(y_test) 
def get_NN_model():
    model = Sequential()
    # model.add(Flatten(input_shape=(1, 1515)))
    model.add(Conv1D(512, 1, activation='relu',input_shape=(1, 1515)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(MaxPooling1D(pool_size=1))

    model.add(Conv1D(128, 1, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(MaxPooling1D(pool_size=1))

    model.add(Conv1D(64, 1, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(MaxPooling1D(pool_size=1))
    model.add(Flatten())

    model.add(Dense(150))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(1))

    model.add(Activation('sigmoid'))


    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
    return model

def write_to_json(model, number):
    
    model_json = model.to_json()
    with open("/home/fatemeh/jan 19/hist/models/model_hist_" + str(number) + ".json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights("/home/fatemeh/jan 19/hist/models/model_hist_" + str(number) + ".h5")


sizes=[5000]#, 10000, 15000, 20000, 25000, 30000, 35000, 40000, 50000, 100000]
for size in sizes:
    print(size)
    X, y = buildSmallerTrainData(XTrainCopy, yTrainCopy, size = size)
    print("finished reading the training data")
    X = np.expand_dims(X, axis=1)
    model = get_NN_model()
    start = time.time()
    model.fit(X, y, epochs = 800, batch_size=128, verbose=0)#, batch_size=64)
    print("finished fiting")
    end = time.time()
    print("Time it took in seconds", end - start)

    write_to_json(model, number=size)
    print(model.summary())
    xT ,yT = buildSmallerTrainData(xTestCopy, yTestCopy, size = 5000)#buildSmallerTestingData(xTrain=xTestCopy, y_train=yTestCopy, size=5000,interval=p*5000)#xTestCopy, yTestCopy#
    xT = np.expand_dims(xT, axis=1)
    y_pred = model.predict(xT)

    accur = 0
    true_pos = 0
    false_pos = 0
    nAll_one = sum(yT)
    nZeros = len(yT) - nAll_one
    for v in range(len(y_pred)):
        if y_pred[v] >= 0.5 and yT[v] == 1:
             accur += 1
             true_pos += 1
        if y_pred[v] < 0.5 and yT[v] == 0:
             accur += 1

        if y_pred[v] >= 0.5 and yT[v] == 0:
            false_pos += 1


    accur = accur/float(len(y_pred))
    print( "batch_ size", 100,"number of training:",size,"Accuracy: ", accur)
    print(false_pos/float(nZeros),true_pos/float(nAll_one))
    


Using TensorFlow backend.


read data:  caida
read data:  httpMultipleTab/4/all
read data:  httpMultipleTab/2/all
read data:  httpMultipleTab/3/all
read data:  httpMultipleTab/1/all
read data:  noisyBitMultipleTab/4/all
read data:  noisyBitMultipleTab/2/all
read data:  noisyBitMultipleTab/3/all
read data:  noisyBitMultipleTab/1/all
read data:  bitcoinCaidaNoise
read data:  bitcoin
5000
finished reading the training data
finished fiting
Time it took in seconds 487.9320032596588
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1, 512)            776192    
_________________________________________________________________
batch_normalization_1 (Batch (None, 1, 512)            2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 512)            0         
_________________________________________________________________
max_pooling1d_1 (M

batch_ size 100 number of training: 15000 Accuracy:  0.8698
0.0028 0.7424
20000
finished reading the training data
finished fiting
Time it took in seconds 2037.589756011963
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_10 (Conv1D)           (None, 1, 512)            776192    
_________________________________________________________________
batch_normalization_13 (Batc (None, 1, 512)            2048      
_________________________________________________________________
dropout_13 (Dropout)         (None, 1, 512)            0         
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 1, 512)            0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 1, 128)            65664     
_________________________________________________________________
batch_normalization_14 (Batc (None,

batch_ size 100 number of training: 30000 Accuracy:  0.915
0.0036 0.8336
35000
finished reading the training data
finished fiting
Time it took in seconds 3529.27294921875
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_19 (Conv1D)           (None, 1, 512)            776192    
_________________________________________________________________
batch_normalization_25 (Batc (None, 1, 512)            2048      
_________________________________________________________________
dropout_25 (Dropout)         (None, 1, 512)            0         
_________________________________________________________________
max_pooling1d_19 (MaxPooling (None, 1, 512)            0         
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 1, 128)            65664     
_________________________________________________________________
batch_normalization_26 (Batc (None, 1

batch_ size 100 number of training: 50000 Accuracy:  0.9028
0.0048 0.8104
100000
finished reading the training data
finished fiting
Time it took in seconds 9833.94884800911
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_28 (Conv1D)           (None, 1, 512)            776192    
_________________________________________________________________
batch_normalization_37 (Batc (None, 1, 512)            2048      
_________________________________________________________________
dropout_37 (Dropout)         (None, 1, 512)            0         
_________________________________________________________________
max_pooling1d_28 (MaxPooling (None, 1, 512)            0         
_________________________________________________________________
conv1d_29 (Conv1D)           (None, 1, 128)            65664     
_________________________________________________________________
batch_normalization_38 (Batc (None,

In [8]:
from keras.models import model_from_json
def load_NN_model(name):
    
    json_file = open("/home/fatemeh/jan 19/hist/models/"+name , 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.load_weights("/home/fatemeh/jan 19/hist/models/"+name[:-5] + ".h5")
    return model

models=os.listdir('/home/fatemeh/jan 19/hist/models/')
for mod in models[0:1]:
    if 'model_hist' in mod and 'json'in mod:
        
        model= load_NN_model(name=mod)  
        X, y = buildSmallerTrainData(XTrainCopy, yTrainCopy, size = 5000)
        X = np.expand_dims(X, axis=1)
        model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
        model.fit(X, y, epochs = 100, batch_size=128, verbose=1)
#         model = load_model("/home/fatemeh/jan 19/hist/models/"+mod[:-5] + ".h5")
        #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        xT ,yT = xTestCopy, yTestCopy#buildSmallerTrainData(xTestCopy, yTestCopy, size = 60000)#buildSmallerTestingData(xTrain=xTestCopy, y_train=yTestCopy, size=5000,interval=p*5000)#xTestCopy, yTestCopy#
        xT = np.expand_dims(xT, axis=1)
        y_pred = model.predict(xT)
        accur = 0
        true_pos = 0
        false_pos = 0
        number_of_all_one = sum(yT)
        nZeros = len(yT) - number_of_all_one
        for v in range(len(y_pred)):
            if y_pred[v] >= 0.5 and yT[v] == 1:
                 accur += 1
                 true_pos += 1
            if y_pred[v] < 0.5 and yT[v] == 0:
                 accur += 1

            if y_pred[v] >= 0.5 and yT[v] == 0:
                false_pos += 1


        accur = accur/float(len(y_pred))
        print(mod, ":   ", false_pos/float(nZeros),true_pos/float(number_of_all_one),
              accur, number_of_all_one,nZeros)

Epoch 1/100
5000/5000 [==============================] - 4s 790us/step - loss: 1.3810e-06 - acc: 1.0000
Epoch 2/100
5000/5000 [==============================] - 1s 143us/step - loss: 2.5469e-04 - acc: 0.9998
Epoch 3/100
5000/5000 [==============================] - 1s 144us/step - loss: 1.9653e-06 - acc: 1.0000
Epoch 4/100
5000/5000 [==============================] - 1s 157us/step - loss: 6.0457e-06 - acc: 1.0000
Epoch 5/100
5000/5000 [==============================] - 1s 153us/step - loss: 5.5074e-06 - acc: 1.0000
Epoch 6/100
5000/5000 [==============================] - 1s 146us/step - loss: 1.5198e-05 - acc: 1.0000
Epoch 7/100
5000/5000 [==============================] - 1s 148us/step - loss: 9.4316e-06 - acc: 1.0000
Epoch 8/100
5000/5000 [==============================] - 1s 150us/step - loss: 7.9512e-05 - acc: 1.0000
Epoch 9/100
5000/5000 [==============================] - 1s 156us/step - loss: 7.1703e-06 - acc: 1.0000
Epoch 10/100
5000/5000 [==============================] - 1s 147

5000/5000 [==============================] - 1s 149us/step - loss: 1.3889e-04 - acc: 1.0000
Epoch 80/100
5000/5000 [==============================] - 1s 148us/step - loss: 1.0313e-04 - acc: 1.0000
Epoch 81/100
5000/5000 [==============================] - 1s 147us/step - loss: 1.0544e-04 - acc: 1.0000
Epoch 82/100
5000/5000 [==============================] - 1s 152us/step - loss: 8.6389e-05 - acc: 1.0000
Epoch 83/100
5000/5000 [==============================] - 1s 148us/step - loss: 1.1764e-04 - acc: 1.0000
Epoch 84/100
5000/5000 [==============================] - 1s 150us/step - loss: 1.0528e-04 - acc: 1.0000
Epoch 85/100
5000/5000 [==============================] - 1s 154us/step - loss: 7.0752e-05 - acc: 1.0000
Epoch 86/100
5000/5000 [==============================] - 1s 153us/step - loss: 7.1297e-05 - acc: 1.0000
Epoch 87/100
5000/5000 [==============================] - 1s 150us/step - loss: 5.3983e-05 - acc: 1.0000
Epoch 88/100
5000/5000 [==============================] - 1s 153us/s